In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections

In [42]:
def collect_losses(root_dir):

    depth = 2

    dict_res_roberta = dict()
    dict_res_camem = dict()
    dict_res_xlm = dict()
    
    for subdir, dirs, files in os.walk(root_dir):
        if subdir[len(root_dir):].count(os.sep) < depth:
            for file_ in files:
                if file_ == "test_results.json":
                    with open(subdir + "/" + file_) as f:
                        d = json.load(f)
                        test_mse = d["test_mse"]
                        test_r2 = d["test_r2"]
                        
                    model_type = subdir.split(os.sep)[-1].split("_")[0]
                    if model_type == "roberta":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_roberta[dataset_name] = {"mse": test_mse, "r2": test_r2}
                    elif model_type == "xlm":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_xlm[dataset_name] = {"mse": test_mse, "r2": test_r2}
                    elif model_type == "camem":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_camem[dataset_name] = {"mse": test_mse, "r2": test_r2}

    return dict_res_roberta, dict_res_camem, dict_res_xlm

def average_en_p_rows(s):
    s = s[[" f" in x for x in s.index]]
    s = s[[" en" in x for x in s.index]]
    s = s[[not "np " in x for x in s.index]]
    return s.mean()

def average_en_np_rows(s):
    s = s[[" f" in x for x in s.index]]
    s = s[[all([x in i for x in ["np ", " en"]]) for i in s.index]]
    return s.mean()

def average_it_p_rows(s):
    s = s[[" f" in x for x in s.index]]
    s = s[[" it" in x for x in s.index]]
    s = s[[not "np " in x for x in s.index]]
    return s.mean()

def average_it_np_rows(s):
    s = s[[" f" in x for x in s.index]]
    s = s[[all([x in i for x in ["np ", " it"]]) for i in s.index]]
    return s.mean()

def process_to_present(df_res, column_width="1.1cm", plot_transpose=True):
    # df_res = df_res.rename(columns=mapping_columns_names)
    if not plot_transpose:
        df_res = df_res.T
    s = df_res.style
    s.format(na_rep='MISS', precision=2)
    print(s.to_latex(column_format='l'+('p{'+f'{column_width}'+'}')*len(df_res.columns)))

In [43]:
res_roberta, res_camem, res_xlm = collect_losses("../output/complexity")

## ROBERTA

In [44]:
df_roberta = pd.DataFrame(res_roberta).T
avg_p = df_roberta.apply(average_en_p_rows)
avg_np = df_roberta.apply(average_en_np_rows)
df_roberta.loc["average p"] = avg_p
df_roberta.loc["average np"] = avg_np

In [45]:
df_roberta

,mse,r2
np f en6,0.181189,0.753015
p f en98,0.143325,0.804629
p f en6,0.156360,0.786860
np nf,0.193794,0.735833
p f en83,0.153382,0.790920
np f en57,0.201341,0.725545
p f en49,0.144864,0.802531
p nf,0.130875,0.821600
p f en57,0.156680,0.786425
np f en83,0.221152,0.698540


In [46]:
process_to_present(df_roberta)

\begin{tabular}{lp{1.1cm}p{1.1cm}}
 & mse & r2 \\
np f en6 & 0.18 & 0.75 \\
p f en98 & 0.14 & 0.80 \\
p f en6 & 0.16 & 0.79 \\
np nf & 0.19 & 0.74 \\
p f en83 & 0.15 & 0.79 \\
np f en57 & 0.20 & 0.73 \\
p f en49 & 0.14 & 0.80 \\
p nf & 0.13 & 0.82 \\
p f en57 & 0.16 & 0.79 \\
np f en83 & 0.22 & 0.70 \\
np f en49 & 0.22 & 0.70 \\
np f en98 & 0.21 & 0.72 \\
average p & 0.15 & 0.79 \\
average np & 0.21 & 0.72 \\
\end{tabular}



## CAMEM

In [52]:
df_camem = pd.DataFrame(res_camem).T
avg_p = df_camem.apply(average_it_p_rows)
avg_np = df_camem.apply(average_it_np_rows)
df_camem.loc["average p"] = avg_p
df_camem.loc["average np"] = avg_np

In [48]:
df_camem

,mse,r2
np f it1,0.377951,0.553839
p nf,0.204893,0.758129
np f it26,0.363298,0.571136
np f it44,0.371652,0.561275
np f it43,0.362287,0.572329
p f it38,0.253111,0.701209
np nf,0.368445,0.565060
p f it43,0.246179,0.709392
p f it44,0.267094,0.684702
p f it26,0.281845,0.667289


In [50]:
process_to_present(df_camem)

\begin{tabular}{lp{1.1cm}p{1.1cm}}
 & mse & r2 \\
np f it1 & 0.38 & 0.55 \\
p nf & 0.20 & 0.76 \\
np f it26 & 0.36 & 0.57 \\
np f it44 & 0.37 & 0.56 \\
np f it43 & 0.36 & 0.57 \\
p f it38 & 0.25 & 0.70 \\
np nf & 0.37 & 0.57 \\
p f it43 & 0.25 & 0.71 \\
p f it44 & 0.27 & 0.68 \\
p f it26 & 0.28 & 0.67 \\
np f it38 & 0.34 & 0.59 \\
p f it1 & 0.24 & 0.72 \\
average p & 0.26 & 0.70 \\
average np & 0.36 & 0.57 \\
\end{tabular}



## XLM

In [58]:
df_xlm = pd.DataFrame(res_xlm).T
en_avg_p = df_xlm.apply(average_en_p_rows)
en_avg_np = df_xlm.apply(average_en_np_rows)
it_avg_p = df_xlm.apply(average_it_p_rows)
it_avg_np = df_xlm.apply(average_it_np_rows)
df_xlm.loc["en average p"] = en_avg_p
df_xlm.loc["en average np"] = en_avg_np
df_xlm.loc["it average p"] = it_avg_p
df_xlm.loc["it average np"] = it_avg_np

In [59]:
df_xlm

,mse,r2
np f en49,0.228629,0.688347
np f it1,0.307056,0.637529
p f en83,0.184099,0.749048
p f en49,0.143438,0.804475
np f en83,0.215541,0.706189
en p nf,0.154085,0.789962
p f en98,0.154193,0.789814
it p nf,0.231441,0.726790
np f en98,0.196709,0.731860
np f en6,0.235940,0.678381


In [60]:
process_to_present(df_xlm)

\begin{tabular}{lp{1.1cm}p{1.1cm}}
 & mse & r2 \\
np f en49 & 0.23 & 0.69 \\
np f it1 & 0.31 & 0.64 \\
p f en83 & 0.18 & 0.75 \\
p f en49 & 0.14 & 0.80 \\
np f en83 & 0.22 & 0.71 \\
en p nf & 0.15 & 0.79 \\
p f en98 & 0.15 & 0.79 \\
it p nf & 0.23 & 0.73 \\
np f en98 & 0.20 & 0.73 \\
np f en6 & 0.24 & 0.68 \\
p f it38 & 0.28 & 0.67 \\
np f it38 & 0.33 & 0.61 \\
np f it26 & 0.32 & 0.62 \\
np f it44 & 0.33 & 0.61 \\
np f it43 & 0.32 & 0.62 \\
p f it1 & 0.26 & 0.69 \\
p f it26 & 0.29 & 0.66 \\
p f it44 & 0.29 & 0.66 \\
p f it43 & 0.27 & 0.68 \\
en np nf & 0.19 & 0.74 \\
p f en57 & 0.16 & 0.78 \\
p f en6 & 0.15 & 0.80 \\
np f en57 & 0.21 & 0.72 \\
it np nf & 0.33 & 0.61 \\
en average p & 0.16 & 0.78 \\
en average np & 0.22 & 0.70 \\
it average p & 0.28 & 0.67 \\
it average np & 0.32 & 0.62 \\
\end{tabular}

